In [8]:
import Gemini
import importlib
from PIL import Image
from Utils import Utils

importlib.reload(Gemini)
model = Gemini.GeminiFlash(cache_refresh=False)

Looking through images...
Done looking through!



In [9]:
system_instruction = """
        You are a data extraction AI. Your task is to extract information from a technical document into a raw JSON object.
        Follow these rules:
            1.  **`id`**: Extract the complete identification string from the document's top header. If there are sub-headers, include them in the ID. This string might contain parentheses, slashes, or other characters.
            2.  **`table`**: Extract the table data in the bottom-left portion of the image. Process this table to create a JSON object.
            *   For each row in the table, use the label from the first column(e.g., 'A', 'A1', 'D/E', 'øb') as a key in your main `table` object.
            *   The value for each key depends on the row's content:
                * **Case 1 (Multiple Columns): ** If the row has values for "MIN", "NOM", and / or "MAX", the value should be a ** nested object ** with `MIN`, `NOM`, and `MAX` as keys. Extract the corresponding numeric values. If a value for MIN, NOM, or MAX is not present or is blank, its value in the JSON must be `null`.
                * **Case 2 (Single Value): ** If the row has only one primary value(like 'D/E' or 'M' in the example), the value should be that single ** number ** directly.
            3. ** Output Format**:
                *   Your entire response MUST be a single, raw JSON object.
                *   Do NOT include any surrounding text, explanations, or markdown formatting like ```json.
                *   The JSON object must have two top-level keys: `id` and `table`.
                *   The value for `id` should be the string extracted from the header.
                *   The value for `table` must be a single JSON object(not an array). The keys of this object are the labels from the table rows.
                *   All extracted text should be trimmed of leading/trailing whitespace.
            4. ** Error Handling**:
                * If the ID is not found, use `"id": null`.
                * If the table is not found, use `"table": {}`.

            Example of required output:
            "CP132/CPG132": {
                "id": "CP132/CPG132",
                "table": {
                    "A": {
                        "MIN": null,
                        "NOM": 1.0,
                        "MAX": 1.1
                    },
                    "A1": {
                        "MIN": 0.15,
                        "NOM": 0.2,
                        "MAX": 0.25
                    },
                    "D/E": {
                        "MIN": 7.9,
                        "NOM": 8.0,
                        "MAX": 8.1
                    },
                    "D1/E1": 6.5,
                    "e": 0.5,
                    "øb": {
                        "MIN": 0.25,
                        "NOM": 0.3,
                        "MAX": 0.35
                    },
                    "ccc": {
                        "MIN": null,
                        "NOM": null,
                        "MAX": 0.1
                    },
                    "ddd": {
                        "MIN": null,
                        "NOM": null,
                        "MAX": 0.08
                    },
                    "eee": {
                        "MIN": null,
                        "NOM": null,
                        "MAX": 0.15
                    },
                    "ZD/ZE": {
                        "MIN": 0.6,
                        "NOM": 0.75,
                        "MAX": 0.9
                    },
                    "M": 14
                }
            }
        """
img = Image.open("/Users/swislar/Desktop/OCR/processed_images/obg0003_page1.jpg")

In [10]:
response = model.model.generate_content([img])
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "{\"id\": \"WB225\", \"table\": {\"A\": {\"MIN\": null, \"NOM\": 3.425, \"MAX\": 3.5}, \"A1\": {\"MIN\": 0.5, \"NOM\": 0.6, \"MAX\": 0.7}, \"D/E\": {\"MIN\": 26.8, \"NOM\": 27.0, \"MAX\": 27.2}, \"D1/E1\": 21.0, \"e\": 1.5, \"\u00f8b\": {\"MIN\": 0.6, \"NOM\": 0.75, \"MAX\": 0.9}, \"ccc\": {\"MIN\": null, \"NOM\": null, \"MAX\": 0.35}, \"M\": 15}}"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 1127,
        "candidates_token_count": 177,
        "total_token_count": 2177
      },
      "model_version": "gemini-2.5-flash"
    }),
)


In [ ]:
input_tokens = model.model.count_tokens([system_instruction, img]).total_tokens
print("Input tokens:", input_tokens)
input_cost = input_tokens * 0.3 / 1_000_000
print("Input cost:", input_cost, '\n')
response = model.model.generate_content([img])
output_tokens = response.usage_metadata.candidates_token_count
print("Output tokens:", output_tokens)
output_cost = output_tokens * 2.5 / 1_000_000
print("Output cost:", output_cost, '\n')
print("Total cost:", input_cost + output_cost)

Input tokens: 1994
Input cost: 0.0005981999999999999 

Output tokens: 201
Output cost: 0.0005025 

Thinking tokens: -37
Thinking cost: -9.25e-05 

Total cost: 0.0010082000000000001


In [ ]:
import json

with open("cache.json", 'r', encoding='utf-8') as f:
    cache = json.load(f)
    
print(len(cache.keys()))

# cached_ids = set(value['image_path'] for value in cache.values())
# print(cached_ids)

In [50]:
from PIL import Image
import Utils
import importlib

importlib.reload(Utils)

img = Image.open(
    "/Users/swislar/Desktop/OCR/all_images/obg0033_page2.jpg")

Utils.Utils.process_image(img).show()

In [49]:
from PIL import Image
import Utils
import regex
import importlib

importlib.reload(Utils)

id = "FF(G)1760 (VIRTEX-5 & 6: XC5VLX220/XC5VLX330/XCE05L33/XC6VLX550T/XC6VLX760)"
xd = "VSVA6865/VSVB6865 (VP1902)"
# Utils.Utils.clean_id(xd)
regex.sub(r'\((?:[^()]|(?R))*\)',"", xd)

'VSVA6865/VSVB6865 '

In [54]:
from PIL import Image
import Utils
import regex
import importlib

importlib.reload(Utils)

id = "CMVA361(SU35P, SU25P, SU10P)"
Utils.Utils.clean_id(id)

'CM361'

In [58]:
from PIL import Image
import Utils
import regex
import importlib

importlib.reload(Utils)

clean_id_1 = "FF1738 ( XC5VFX130T/XC5VFX200T/XQ5VFX200T/XC5VLX220T/XCE05L22T/XC5VSX240T/XQ5VSX240T/XC5VLX330T)"
target_id = "FF1738 (XCSVFX130T/XCSVFX200T/XOSVFX200T/XCSVLX220T/XCEOSL22T/XCSVSX240T/ XOSVSX240T/XCSVLX330T)"

Utils.Utils.similarity_score(target_id, clean_id_1)

FF1738XCSVFX130TXCSVFX200TXOSVFX200TXCSVLX220TXCEOSL22TXCSVSX240TXOSVSX240TXCSVLX330T
FF1738XC5VFX130TXC5VFX200TXQ5VFX200TXC5VLX220TXCE05L22TXC5VSX240TXQ5VSX240TXC5VLX330T


87.05882352941177

In [64]:
from rapidfuzz import fuzz

clean_code_1 = re.sub(r'[^a-zA-Z0-9]', '', clean_id_1)
clean_code_2 = re.sub(r'[^a-zA-Z0-9]', '', target_id)
for i in range(len(clean_code_1)):
    if (clean_code_1[i] != clean_code_2[i]):
        print(i, clean_code_1[i], clean_code_2[i])
fuzz.ratio(clean_code_1, clean_code_2)

8 5 S
18 5 S
27 Q O
28 5 S
38 5 S
49 0 O
50 5 S
57 5 S
66 Q O
67 5 S
77 5 S


87.05882352941177